# 06. Object-Oriented Patterns in Kotlin

디자인 패턴에 대해 소개하는 그런 장인데 ... 솔직히 이 장은 그렇게 도움이 되지는 않을 듯

최대한 간단하게 여러가지 디자인 패턴을 소개하려 해서 모르는 사람은 이게뭔가 싶고 아는 사람은 그냥 너무 당연한 예시 같을거고 뭐 그런 느낌이다

그래서 책의 내용 대신 디자인 패턴과 관련된 것을 한두 개 소개하는 게 더 더 도움이 될 거 같다.

## Template method
- https://asvid.github.io/kotlin-template-method

이 노트북은 위 블로그 글에 나오는 코드를 기본 바탕으로 하여 약간씩 적절히 변형한 것이다.

참고로, 피자 예제는 이 블로그 글 말고도 다른 OOP관련 책이나 자료 등에서도 데코레이터 등의 패턴을 설명할 때도 단골로 사용하는 테마의 예제임

template method를 이해하면 factory method도 생성자에 대한 template method에 해당하는 개념이고, 또 책에 나오는 strategy도 template method의 특별한 사례로 볼 수도 있으니 ...

***주의***: 이런 피자 예제 같은 것들이 설명하는 디자인 패턴을 적용하는 것이 매우 적합한 상황이라고 오해하면 안된다.
그냥 그 개념을 설명하기 위해 OOP를 실무에서 본격적으로 많이 해보지 않은 사람도 이해하기 좋은 주제로 예제를 작성하는 것.

### Pizza

In [1]:
abstract class Pizza { // base class for all pizza types

    // 피자 만들기 template method
    fun make() { // steps are the same for every pizza
        makeDough()      // 피자도우 반죽
        applySauce()     // 소스 바르기
        addIngredients() // 토핑 재료 추가
        bake()           // 굽기
    }

    // default implementation for each step
    // concrete classes needs to override only the distinctive ones
    open fun makeDough() = println("making 30cm dough")
    open fun addIngredients() = println("adding cheese")
    open fun applySauce() = println("applying tomato sauce")
    open fun bake() = println("baking for 20 minutes")
}

In [3]:
open class Pepperoni : Pizza() { // concrete type of pizza
    override fun addIngredients() { // overriding ingredients according to recipe
        println("adding salami")
        println("adding onion")
        println("adding cheese")
    }
}

// 우리나라는 보통 R L F 이렇게 가니까 (코스트코같이 그런 미국사이즈 피자 다루는 대형마트 말고는)
// dough size variation
class PepperoniRegular: Pepperoni() { override fun makeDough() = println("making 23cm dough") }
class PepperoniLarge:   Pepperoni() // default size dough
class PepperoniFamily:  Pepperoni() { override fun makeDough() = println("making 38cm dough") }

// but it's not controlling the process of making a pizza
// all other methods are left with default implementation

In [5]:
PepperoniRegular().make()

making 23cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


In [6]:
PepperoniLarge().make()

making 30cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


In [7]:
PepperoniFamily().make()

making 38cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


### Pizza lambdas

In [12]:
abstract class Pizza( // base class constructor is taking lambdas but provides default implementation
    val makeDough:      () -> Unit = { println("making 30cm dough") },
    val applySauce:     () -> Unit = { println("applying tomato sauce") },
    val addIngredients: () -> Unit = { println("adding cheese") },
    val bake:           () -> Unit = { println("baking for 20 minutes") }
) {
    fun make() { // unchanged template method
        makeDough() // calling lambda parameter from constructor that can be replaced
        applySauce()
        addIngredients()
        bake()
    }
}

In [14]:
val pepperoniAddons: () -> Unit = {
    println("adding salami")
    println("adding onion")
    println("adding cheese")
}

class PepperoniRegular: Pizza(
    addIngredients = pepperoniAddons,
    makeDough = { println("making 23cm dough") }
)
class PepperoniLarge:   Pizza(  // default size dough
    addIngredients = pepperoniAddons
)
class PepperoniFamily:  Pizza(
    addIngredients = pepperoniAddons,
    makeDough = { println("making 38cm dough") }
)

In [15]:
PepperoniRegular().make()

making 23cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


In [16]:
PepperoniLarge().make()

making 30cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes


In [17]:
PepperoniFamily().make()

making 38cm dough
applying tomato sauce
adding salami
adding onion
adding cheese
baking for 20 minutes
